based on https://blog.langchain.dev/constructing-knowledge-graphs-from-text-using-openai-functions/

first run neo4j docker 

docker run \
    -p 7474:7474 -p 7687:7687 \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4J_PLUGINS=\[\"apoc\"\] \
    --env NEO4J_AUTH=neo4j/password \
    --volume=/media/jm/hddData/datasets/neo4j_data:/data \
    neo4j:5.19.0






open on http://localhost:7474/browser/
set password to "password"



in the end MATCH (n) RETURN n

In [1]:
from langchain.graphs import Neo4jGraph
from neo4j import GraphDatabase
from langchain.prompts import ChatPromptTemplate
from tqdm import tqdm
from langchain.chains import GraphCypherQAChain
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
url = "neo4j://localhost:7687"
# url = "neo4j://localhost:7474"
username ="neo4j"
password = "password"

os.environ["NEO4J_URI"] = url #"bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] =username # "neo4j"
os.environ["NEO4J_PASSWORD"] = password #"password"


graph = Neo4jGraph()



QUERY = """
MATCH (keanu:Person {name:'Keanu Reeves'})
RETURN keanu.name AS name, keanu.born AS born
"""
# QUERY = """
# "MATCH (m:Movie)-[:IN_GENRE]->(:Genre {name:$genre})
# RETURN m.title, m.plot
# ORDER BY m.imdbRating DESC LIMIT 5"
# """

graph.query(QUERY)
# # # { host: '127.0.0.1', path: '/', port: 7689 }


[]

In [ ]:
llm_med = ChatOllama(model="medllama2")

In [2]:
from langchain.graphs import Neo4jGraph
from neo4j import GraphDatabase
url = "neo4j://localhost:7687"
username ="neo4j"
password = "password"

def _create_and_return_greeting(tx, message):
    result = tx.run("CREATE (a:Greeting) "
                    "SET a.message = $message "
                    "RETURN a.message + ', from node ' + id(a)", message=message)
    return result.single()[0]

message="hello, world"
driver=GraphDatabase.driver(url, auth=(username, password))
with driver.session() as session:
    greeting = session.execute_write(_create_and_return_greeting, message)
    print(greeting)

hello, world, from node 0


In [13]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
llm = ChatAnthropic(temperature=0, model_name="claude-3-opus-20240229")

llm_transformer = LLMGraphTransformer(llm=llm)

In [14]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

# messages = [
#     SystemMessage(
#         content="You are a helpful assistant that translates English to French."
#     ),
#     HumanMessage(
#         content="Translate this sentence from English to French. I love programming."
#     ),
# ]
# llm.invoke(messages)

In [6]:
p="/workspaces/thyroid_knowledge_graph/data/joined.txt"
with open(p, 'r') as file:
    text = file.read()

text

'Thyroid cancer is divided into differentiated thyroid cancer (DTC) and undifferentiated thyroid cancer. In contrast to the undifferentiated thyroid cancer, the vast majority of the DTC accumulates iodine in the tumour cells. Utilizing this characteristic, DTC patients can undergo radioiodine therapy (RIT). RIT of DTC patients provides a very effective therapy with only minimal side effects using standard activities, which is established for more than half a century. Patients are treated according to their tumour stage. Low-risk patients usually undergo a single RIT with activities up to 4 GBq 131 I. High-risk patients undergo at least one or several RIT with higher activities [1,2]. While performing RIT with high activities of 131 I, the organs at risk need specific attention. These are the bone marrow and the lung, especially in case of multiple metastases. The amount of 131 I activity and the number of RIT depend on the results of the first RIT: if metastases were detected in the po

In [24]:
from langchain_core.documents import Document
import os
import json
from semantic_text_splitter import TextSplitter
max_characters = 500
# Optionally can also have the splitter not trim whitespace for you
splitter = TextSplitter()

# Document(page_content=sect['text'])

folder_path = "/workspaces/thyroid_knowledge_graph/preprocessed"
file_paths = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)

res=[]
for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        sections= data['sections']
        for sect in sections:
            # res.append(sect['text'])
            for chunk in splitter.chunks(sect['text'], max_characters):

                res.append(chunk)
# res="\n".join(res)   
# res_path="/workspaces/thyroid_knowledge_graph/data/joined.txt"

# with open(res_path, 'w') as file:
#     file.write(res)


# text="""Thyroid cancer is divided into differentiated thyroid cancer (DTC) and undifferentiated thyroid cancer. In contrast to the undifferentiated thyroid cancer, the vast majority of the DTC accumulates iodine in the tumour cells. Utilizing this characteristic, DTC patients can undergo radioiodine therapy (RIT). RIT of DTC patients provides a very effective therapy with only minimal side effects using standard activities, which is established for more than half a century. Patients are treated according to their tumour stage. Low-risk patients usually undergo a single RIT with activities up to 4 GBq 131 I. High-risk patients undergo at least one or several RIT with higher activities [1,2]. While performing RIT with high activities of 131 I, the organs at risk need specific attention"""
# text="""Thyroid cancer is divided into differentiated thyroid cancer (DTC) and undifferentiated thyroid cancer. In contrast to the undifferentiated thyroid cancer, the vast majority of the DTC accumulates iodine in the tumour cells. Utilizing this characteristic, DTC patients can undergo radioiodine therapy (RIT). RIT of DTC patients provides a very effective therapy with only minimal side effects using standard activities, which is established for more than half a century. Patients are treated according to their tumour stage. Low-risk patients usually undergo a single RIT with activities up to 4 GBq 131 I. High-risk patients undergo at least one or several RIT with higher activities [1,2]. While performing RIT with high activities of 131 I, the organs at risk need specific attention. These are the bone marrow and the lung, especially in case of multiple metastases. The amount of 131 I activity and the number of RIT depend on the results of the first RIT: if metastases were detected in the posttherapeutic scan, further RIT may be necessary for an effective treatment of metastases. Therefore, a pretherapeutic diagnostic, which detects iodine avid metastases, is desirable. The same holds for the first RIT for patients with a high likelihood for metastases. The pretherapeutic risk stratification is one of the greatest challenges in treating DTC patients. Although the overall prognosis for the most DTC patients is excellent [3], unfortunately, some DTC patients suffer from recurrences or persistent disease. These patients have already undergone RIT. In cases of recurrent or persistent disease, the cancer cells often show less up to none radioiodine uptake. Patients with low or missing radioiodine uptake would not or would only slightly benefit from an additional RIT [4]. Therefore, the knowledge of the localization and the radioiodine uptake level of lesions have great influence on therapy decision and the amount of therapeutic activity.
# Pretherapeutic imaging of DTC patients can be performed with different radioiodine isotopes and the corresponding imaging modalities. 131 I as diagnostic isotope is used since decades. Usually, activities up to 185 MBq 131 I are administered [5]. Another radioiodine isotope for pretherapeutic imaging is 123 I, which is hardly used due to its lower imaging sensitivity and higher costs [5]. Both isotopes can be visualized by planar scintigraphies with a gamma camera or as 3D imaging with a single photon emission tomography (SPECT) camera. Compared to positron emission tomography (PET), the spatial resolution of gamma cameras is lower. However, radioiodine diagnostics using 131 I prior to intended RIT is not advisable mainly for two reasons. First, until today it is not possible to derive reliable lesion dosimetry data from 131 I scans. Second, 131 I imaging needs relative high activities resulting in high radiation doses. It is well known, that this leads to a so called stunning and, thus, reduces the response of subsequent RIT [6]. 124 I PET in combination with computed tomography (CT) overcomes both problems. It offers optimal imaging to detect radioiodine uptake in tumour cells before patients undergo RIT, while only low 124 I activities are required compared to 131 I imaging [7][8][9][10][11]. Stunning is unlikely and was not reported yet [5]. In addition, pretherapeutic 124 I diagnostic provides, in comparison with 131 I and 123 I, a higher quantitative capacity to perform a reliable lesion dosimetry. This enables a tailor-made RIT with optimized absorbed tumour doses and an 131 I activity, which is considered to be safe for the patient, based on the blood dosimetry.
# There are two important requirements for a successful performance of124 I PET(/CT) diagnostics or RIT: First, all patients are put on a low iodine diet. This is important to increase the radioiodine uptake of the tumour cells. After performing a CT scan with contrast agents containing iodine, for example, the tumour cells are saturated with iodine and would show only a low radioiodine uptake. Therefore, any iodine contamination has to be avoided to ensure a good radioiodine uptake during diagnostic imaging or therapy. Second, the tumour cells have to be stimulated to achieve a high radioiodine uptake. The increase of the thyroid stimulating hormone (TSH) can be reached on two ways: endogenous or exogenous stimulation. The endogenous stimulation is usually chosen after initial thyroidectomy. Patients have to stay on hormone withdrawal for about 4 weeks, until the TSH serum concentration exceeds 30 mU/l [2,5]. During this time, patients may experience hypothyroid symptoms such as fatigue, listlessness, depression and concentration disorders. The other way is an exogenous stimulation with recombinant human TSH (rhTSH). The rhTSH is administered via an intramuscular injection two and one day before diagnostic imaging or treatment. The exogenous stimulation spares patients having the hypothyroid symptoms. Moreover, a lower blood dose was reported in rhTSH stimulated patients [12,13] compared to endogenous stimulation. However, the equivalence of therapeutic efficiency concerning tumour elimination under rhTSH stimulation is not proven. 124 I is a positron emitting nuclide with a half-life of 4.2 days. Like many other positron emitting nuclides, 124 I is produced in a cyclotron. Under endogenous or exogenous TSH stimulation, 124 I can be administered in two different ways: orally or intravenously. The amount of administered 124 I activities reported in the literature is between 25 MBq and 74 MBq [7,14,15].
# In the time of combined PET/CT scanners, it is much easier to locate focal radioiodine uptake for example in the neck [16]. The new scanner generation provides fast scans with an acquisition time of e.g. 15 min for a scan from thigh to head. Besides the scan parameters and the scanner properties, the time point of scan is very important. In the beginning of 124 I PET(/CT) imaging, multiple scans were performed at 4, 24, 48, 72, and ≥96 h after 124 I application. This time and resource consuming approach has been optimized and the necessary scans could be reduced to 2 time points: 24 and 96 h after 124 I application [14]. The first scan 24 h after 124 I application enables the detection of focal pathological iodine uptake as correlate for metastases or local relapse. This scan is sufficient for diagnostic purpose. The first and second scans are mainly needed for lesion dosimetry to estimate the lesion absorbed (radiation) dose during RIT (details see below). However, occasionally, weakly radioiodine accumulating metastases only show uptake in the late scan.
# The RIT is crucial for an effective treatment of DTC [17,18]. Patients with higher initial tumour stage or advanced disease with lymph node or distant metastases routinely undergo RIT with higher 131 I activities or several RIT. The knowledge of metastases prior to first RIT would be beneficial for therapy planning. If no metastases are present, lower activities of 131 I would be reasonable. Patients with many metastases, especially bone or pulmonary metastases, should be treated carefully because of bone marrow toxicity and the risk of radiation-related pneumonitis or lung fibrosis [19,20].
# Another advantage of pretherapeutic 124 I PET is the re-staging and especially up-staging in patients with suspect findings in sonography. Shortly before RIT (about three weeks after thyroidectomy) patients should undergo sonography of the neck. If conspicuous lymph nodes are present, additional clarification is necessary to adapt therapy management. In this process, 124 I PET offers the possibility to detect focal iodine uptake in case of lymph node metastases and therapy activity of 131 I can be adapted. If no radioiodine uptake was found in conspicuous lymph nodes, it might be a reactive lymph node or a radioiodine negative lymph node metastasis; both will not be affected by the RIT.
# During the first RIT, there is usually focal radioiodine uptake in the thyroid bed in accordance with thyroid remnant tissues. Other focal radioiodine uptake located beyond the thyroid bed is suspicious for malignancy, for example in the neck is typical for lymph node metastases.
# In the clinical course of patients with DTC, some patients suffer from tumour persistence after initial RIT or experience a relapse of DTC at a later stage. These patients mostly present an elevated and/or increasing serum level of thyroglobulin. Sometimes new tissue in the thyroid bed or suspect lymph nodes can be detected in sonography of the neck as a correlate. The most preferable therapy option would be an additional RIT in these cases. Unfortunately, thyroid cancer cells can dedifferentiate and partially or completely lose the ability to accumulate iodine. These patients with the so-called radioiodine negative metastases do not or only slightly profit from additional RIT. Other therapies such as surgery or external beam radiotherapy would be the therapy of choice in radioiodine negative metastases. But not all patients are radioiodine negative in case of recurrent disease. Therefore, a selection must be performed prior to futile RIT.
# After initial RIT and thyroid remnant ablation, no pathological focal iodine uptake should be visible. Therefore, every focal uptake, for example in the neck or lung, is highly suspect for metastases or local relapse (except physiological uptake, e.g. in the salivary glands).
# Within the last years, simultaneous PET/MRI (PET/magnetic resonance imaging) has been successfully implemented into scientific and clinical imaging, demonstrating its excellent diagnostic potential, mainly in oncologic imaging. Based on its excellent soft tissue contrast, MRI is known to provide superior assessment of head and neck tumours and with the increasing implementation of integrated whole-body PET/MR devices, combining the strength of PET as well as morphological and functional MRI. In this assessment, it could be of great advantage in the localization of focal iodine uptake. The first publication about simultaneous 124 I PET/MRI showed promising results [21].
# The 124 I dosimetry deals with the determination of tumour lesion dose per administered 131 I activity (LDpA) and the estimation of the so-called maximum tolerable activity (MTA) of 131 I. The LDpA can help in the decision process if an additional RIT is a promising approach. If a prescribed lesion dose seems to be reachable, the RIT is combined with a significant chance of cure [22][23][24]. Due to the heterogeneity of metastases of a patient, there may be a significant spread of LDpA in the different metastases. In consequence, RIT might be helpful in some metastases and the other metastases need to be treated by surgery or external beam radiotherapy.
# The MTA derived from blood dosimetry is the 131 I activity that results in 2 Gy blood dose, which is used as a surrogate for the bone marrow dose. Blood doses below 2 Gy are expected to be tolerable without relevant side effects [19]. Furthermore, it is possible to evaluate the risk of radiation-related fibrosis in patients with or without disseminated lung metastases [19,20]. The blood dosimetry is based on blood sample (BS) and whole-body counter (WC) measurements (see Figures 123).   The lesion and blood dosimetry consists of measurements at different time points after application of 124 I to determine the iodine kinetics. The lesion dosimetry requires a minimum of two PET(/CT) scans at 24 and ≥96 h after 124 I application. The blood dosimetry by the EANM dosimetry committee uses a comprehensive protocol with measurements over at least 4 days [25], which was optimized and shortened by Jentzen et al. [26]. For the optimized blood dosimetry protocols, patients are divided into the groups prior to first RIT and after first RIT. Prior to initial RIT, more measurements are recommended due to the presence of thyroid tissue. After the first RIT, only three time points are sufficient. In Figure 4, the required measurements are visualized. Further details on the calculations of lesion and blood dosimetry can be found in Jentzen et al. [14,26] (see examples in Figures 5 and6).  
# """
documents = list(map(lambda t: Document(page_content=t), res))
# documents


documents

[Document(page_content='Thyroid cancer is divided into differentiated thyroid cancer (DTC) and undifferentiated thyroid cancer. In contrast to the undifferentiated thyroid cancer, the vast majority of the DTC accumulates iodine in the tumour cells. Utilizing this characteristic, DTC patients can undergo radioiodine therapy (RIT). RIT of DTC patients provides a very effective therapy with only minimal side effects using standard activities, which is established for more than half a century.'),
 Document(page_content='Patients are treated according to their tumour stage. Low-risk patients usually undergo a single RIT with activities up to 4 GBq 131 I. High-risk patients undergo at least one or several RIT with higher activities [1,2]. While performing RIT with high activities of 131 I, the organs at risk need specific attention. These are the bone marrow and the lung, especially in case of multiple metastases.'),
 Document(page_content='The amount of 131 I activity and the number of RIT 

In [34]:
print(documents[1].page_content)
len(documents[1].page_content)

Patients are treated according to their tumour stage. Low-risk patients usually undergo a single RIT with activities up to 4 GBq 131 I. High-risk patients undergo at least one or several RIT with higher activities [1,2]. While performing RIT with high activities of 131 I, the organs at risk need specific attention. These are the bone marrow and the lung, especially in case of multiple metastases.


399

In [43]:
# text="""Thyroid cancer is divided into differentiated thyroid cancer (DTC) and undifferentiated thyroid cancer. In contrast to the undifferentiated thyroid cancer, the vast majority of the DTC accumulates iodine in the tumour cells. Utilizing this characteristic, DTC patients can undergo radioiodine therapy (RIT). RIT of DTC patients provides a very effective therapy with only minimal side effects using standard activities, which is established for more than half a century. Patients are treated according to their tumour stage. Low-risk patients usually undergo a single RIT with activities up to 4 GBq 131 I. High-risk patients undergo at least one or several RIT with higher activities [1,2]. While performing RIT with high activities of 131 I, the organs at risk need specific attention"""
index=0
for i in range(16,30):
    try:
        graph_documents = llm_transformer.convert_to_graph_documents([documents[i]])
        graph.add_graph_documents(graph_documents)
    except:
        index=index+1
        print(f"error {index}")
        pass


error 1


In [37]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='131 I Activity', type='Measurement'), Node(id='Rit', type='Treatment'), Node(id='Metastases', type='Disease'), Node(id='Posttherapeutic Scan', type='Diagnostic'), Node(id='Pretherapeutic Diagnostic', type='Diagnostic'), Node(id='Iodine Avid Metastases', type='Disease'), Node(id='Dtc Patients', type='Person')]
Relationships:[Relationship(source=Node(id='Rit', type='Treatment'), target=Node(id='131 I Activity', type='Measurement'), type='DETERMINES'), Relationship(source=Node(id='Posttherapeutic Scan', type='Diagnostic'), target=Node(id='Metastases', type='Disease'), type='DETECTS'), Relationship(source=Node(id='Pretherapeutic Diagnostic', type='Diagnostic'), target=Node(id='Iodine Avid Metastases', type='Disease'), type='DETECTS'), Relationship(source=Node(id='Rit', type='Treatment'), target=Node(id='Dtc Patients', type='Person'), type='TREATS')]


to see in neo4j all nodes just type MATCH (n) RETURN n

In [ ]:
# llm_med = ChatOllama(model="medllama2")
# cypher_chain = GraphCypherQAChain.from_llm(
#     graph=graph,
#     cypher_llm=llm_med,
#     qa_llm=llm_med,
#     validate_cypher=False, # Validate relationship directions
#     verbose=False
# )


# aaaaa=cypher_chain.invoke({"query": "Who is Marie Curie?"})
# aaaaa


In [ ]:
#from https://blog.langchain.dev/using-a-knowledge-graph-to-implement-a-devops-rag-application/

In [ ]:
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector


from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "gpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
# embedding = hf.embed_query("hi this is harrison")


vector_index = Neo4jVector.from_existing_graph(
    hf,
    url=url,
    username=username,
    password=password,
    index_name='tasks',
    node_label="Task",
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
)

In [ ]:
from langchain.chains import RetrievalQA
llm_med = ChatOllama(model="medllama2")
vector_qa = RetrievalQA.from_chain_type(
    llm=llm_med,
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)
vector_qa.run(
    "Who is Marie Curie?"
)

In [ ]:
response = vector_index.similarity_search(
    "Who is Marie Curie?"
)
# print(response[0].page_content)
print(response)